In [ ]:
import geopandas
gdf = geopandas.read_file('data/pocndvi.shp')
# find rows where geometry is MULTIPOLYGON
gdf = gdf[gdf.geometry.type != 'MultiPolygon']

In [ ]:
gidIds = gdf['gisId'].unique()

polygons = []

for gisId in gidIds:
    mask = gdf['gisId'] == gisId
    polygon = gdf[mask]['geometry'].__geo_interface__["features"][0]["geometry"]
    polygons.append(polygon)

# split into two lists
polygons1 = polygons[:len(polygons)//2]
polygons2 = polygons[len(polygons)//2:]
len(polygons1), len(polygons2)

In [ ]:
import sys 
import os
sys.path.append(os.path.abspath("/Users/tammerkamel/dev/proj/Streambatch/src/Streambatch/"))
from module1 import StreambatchConnection

In [ ]:
connection = StreambatchConnection(open('key.txt').read().strip())
query_id1 = connection.request_ndvi(
            polygons=polygons1,
            sources=['ndvi.savgol'],
            aggregation='mean',
            start_date='2022-01-01',
            end_date='2023-08-31',
            query_id='5af18e61-f3d9-4390-aa7e-f3a87f5303b3'
            )

query_id2 = connection.request_ndvi(
            polygons=polygons2,
            sources=['ndvi.savgol'],
            aggregation='mean',
            start_date='2022-01-01',
            end_date='2023-08-31',
            query_id='b3f4c5f3-c5c5-4a25-bdc7-ead7e5b0b417'
            )

In [ ]:
df1 = connection.get_data(query_id1)
df2 = connection.get_data(query_id2)

In [ ]:
import pandas as pd
# split gidIds into two lists
gidIds1 = gidIds[:len(gidIds)//2]
gidIds2 = gidIds[len(gidIds)//2:]

locations1 = df1['location']
locations2 = df2['location']
df1['cyan id'] = gidIds1[df1['location']]
df2['cyan id'] = gidIds2[df2['location']]
# remove location column
df1 = df1.drop(columns=['location'])
df2 = df2.drop(columns=['location'])
# concat using pd.concat
df = pd.concat([df1, df2], axis=0).copy()
# make cyan id the first column
df = df[['cyan id'] + [col for col in df.columns if col != 'cyan id']]
# remove colmns ndvi.interpolated, ndvi.sentinel2, ndvi.landsat
df = df.drop(columns=['ndvi.interpolated', 'ndvi.sentinel2', 'ndvi.landsat','polygon'])
# rename column ndvi.savgol as ndvi
df = df.rename(columns={'ndvi.savgol': 'ndvi'})

# save df as parquet
df.to_parquet('data/ndvi_for_cyan.parquet')

df.head()



In [ ]:
import matplotlib.pyplot as plt

# for l in df['cyan id'].unique():
for l in ['24094']:
    df1 = df[df['cyan id'] == l]

    # scatter plot time vs ndvi.savgol with s=3 point size
    # plt.figure(figsize=(30, 15))

    plt.figure(figsize=(30, 15))
    plt.title(f'cyan id: {l}')
    plt.rcParams.update({'font.size': 22})

    plt.scatter(df1['time'], df1['ndvi'], label='savgol', s=13)
    # plt.scatter(df1['time'], df1['ndvi.landsat'], label='landsat',s=63,color='green')
    # plt.scatter(df1['time'], df1['ndvi.sentinel2'], label='sentinel-2',s=63,color='red')

    plt.grid(True)
    plt.ylim(0, 1.)

    plt.xlabel('time')
    plt.ylabel('ndvi')
    plt.legend()



plt.show()